In [564]:
import tensorflow as tf
import cirq
import numpy as np
import random
import sympy
from math import pi
import pandas as pd
%matplotlib inline

np.random.seed(42)

In [553]:
def U_phi(q, W):
    for i in range(len(q)):
        rot = cirq.ZPowGate(exponent=W[i])
        yield rot(q[i])
    for i in range(len(q)-1):
        for j in range(i+1,len(q)):
            rot = cirq.ZPowGate(exponent=(pi-W[i])*(pi-W[j]))
            yield rot.on(q[j]).controlled_by(q[i])
            
def fancy_U(q, W):
    for i in range(len(q)):
        yield cirq.H(q[i])
    yield U_phi(q, W)
    for i in range(len(q)):
        yield cirq.H(q[i])
    yield U_phi(q, W)

def W_theta(q, theta):
    for i in range(len(q)):
        yield cirq.CZ.on(q[(i+1)%len(q)],q[i])
    for i in range(len(q)):
        rot = cirq.ZPowGate(exponent=theta[i])
        yield rot(q[i])

def measure(q):
    for i in range(len(q)):
        yield cirq.measure(q[i], key=str(i))

In [550]:
def accuracy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)
    return loss

def optimizer(loss, theta):
    print(loss,theta)
    return 0.9*np.array(theta) + 0.01 * np.array(loss)#nabla * loss

In [551]:
def circuit(q, W, theta):
    yield fancy_U(q, W)
    for i in range(len(theta)):
        if i == 0:
            for j in range(len(q)):
                rot = cirq.ZPowGate(exponent=theta[i][j])
                yield rot(q[j])
        else:
            yield W_theta(q, theta[i])
    yield measure(q)

In [568]:
nr_qubits = 3
nr_layers = 2
batch_size = 5
repetitions = 100

#Set up input and output qubits.
qubits = [cirq.GridQubit(i, 0) for i in range(nr_qubits)]

df = pd.read_csv("QA_data.csv")
X = df.iloc[:,:3].to_numpy()
Y = df.iloc[:,3].to_numpy()

init_theta = [np.random.randn(nr_qubits) for i in range(nr_layers+1)]
theta = init_theta
optimiser = tf.train.AdamOptimizer()

for iter in range(5):
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    print("iteration:",iter)

    simulator = cirq.Simulator()
    loss = 0
    count = [0 for i in range(batch_size)]
    loss = [0 for i in range(batch_size)]
    labels = [[0,0] for i in range(batch_size)]
    for i in range(len(X_batch)):
        c = cirq.Circuit()
        c.append(circuit(qubits, X_batch[i], init_rot))
        for k in range(repetitions):
            results = simulator.run(c)
            for j in range(nr_qubits):
                count = results.histogram(key=str(j))[0]
            if count % 2 == 0:
                labels[i][0] += 1
            else:
                labels[i][1] += 1
        print("Data point:",Y_batch[i],"Labels {0,1}:",labels[i])
print('Circuit:')
print(c)

iteration: 0
Data point: 1 Labels {0,1}: [54, 46]
Data point: 0 Labels {0,1}: [55, 45]
Data point: 1 Labels {0,1}: [36, 64]
Data point: 0 Labels {0,1}: [32, 68]
Data point: 1 Labels {0,1}: [58, 42]
iteration: 1
Data point: 0 Labels {0,1}: [70, 30]
Data point: 0 Labels {0,1}: [45, 55]
Data point: 0 Labels {0,1}: [62, 38]
Data point: 0 Labels {0,1}: [66, 34]
Data point: 0 Labels {0,1}: [35, 65]
iteration: 2
Data point: 0 Labels {0,1}: [33, 67]
Data point: 1 Labels {0,1}: [56, 44]
Data point: 0 Labels {0,1}: [61, 39]
Data point: 1 Labels {0,1}: [38, 62]
Data point: 0 Labels {0,1}: [46, 54]
iteration: 3
Data point: 0 Labels {0,1}: [45, 55]
Data point: 0 Labels {0,1}: [85, 15]
Data point: 0 Labels {0,1}: [75, 25]
Data point: 0 Labels {0,1}: [57, 43]
Data point: 0 Labels {0,1}: [36, 64]
iteration: 4
Data point: 0 Labels {0,1}: [69, 31]
Data point: 0 Labels {0,1}: [12, 88]
Data point: 0 Labels {0,1}: [74, 26]
Data point: 0 Labels {0,1}: [31, 69]
Data point: 1 Labels {0,1}: [46, 54]
Circuit:
 

In [460]:
# def rot_x_layer(length, half_turns):
#     """Yields X rotations by half_turns on a square grid of given length."""
#     rot1 = cirq.ZPowGate(exponent=half_turns)
#     rot2 = cirq.XPowGate(exponent=half_turns)
#     rot3 = cirq.ZPowGate(exponent=half_turns)
#     for i in range(length):
#         for j in range(length):
#             yield rot(cirq.GridQubit(i, j))

# circuit = cirq.Circuit()
# circuit.append(rot_x_layer(2, 0.1))
# print(circuit)

In [432]:
# def rand2d(rows, cols):
#     return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

# def random_instance(length):
#     # transverse field terms
#     h = rand2d(length, length)
#     # links within a row
#     jr = rand2d(length - 1, length)
#     # links within a column
#     jc = rand2d(length, length - 1)
#     return (h, jr, jc)

# h, jr, jc = random_instance(3)
# print('transverse fields: {}'.format(h))
# print('row j fields: {}'.format(jr))
# print('column j fields: {}'.format(jc))

In [433]:
# def rot_z_layer(h, half_turns):
#     """Yields Z rotations by half_turns conditioned on the field h."""
#     gate = cirq.ZPowGate(exponent=half_turns)
#     for i, h_row in enumerate(h):
#         for j, h_ij in enumerate(h_row):
#             if h_ij == 1:
#                 yield gate(cirq.GridQubit(i, j))

# def rot_11_layer(jr, jc, half_turns):
#     """Yields rotations about |11> conditioned on the jr and jc fields."""
#     gate = cirq.CZPowGate(exponent=half_turns)    
#     for i, jr_row in enumerate(jr):
#         for j, jr_ij in enumerate(jr_row):
#             if jr_ij == -1:
#                 yield cirq.X(cirq.GridQubit(i, j))
#                 yield cirq.X(cirq.GridQubit(i + 1, j))
#             yield gate(cirq.GridQubit(i, j),
#                        cirq.GridQubit(i + 1, j))
#             if jr_ij == -1:
#                 yield cirq.X(cirq.GridQubit(i, j))
#                 yield cirq.X(cirq.GridQubit(i + 1, j))

#     for i, jc_row in enumerate(jc):
#         for j, jc_ij in enumerate(jc_row):
#             if jc_ij == -1:
#                 yield cirq.X(cirq.GridQubit(i, j))
#                 yield cirq.X(cirq.GridQubit(i, j + 1))
#             yield gate(cirq.GridQubit(i, j),
#                        cirq.GridQubit(i, j + 1))
#             if jc_ij == -1:
#                 yield cirq.X(cirq.GridQubit(i, j))
#                 yield cirq.X(cirq.GridQubit(i, j + 1))
                
# def one_step(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
#     length = len(h)
#     yield rot_x_layer(length, x_half_turns)
#     yield rot_z_layer(h, h_half_turns)
#     yield rot_11_layer(jr, jc, j_half_turns)

In [434]:
# h, jr, jc = random_instance(3)

# circuit = cirq.Circuit()    
# circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3),
#                strategy=cirq.InsertStrategy.EARLIEST)
# print(circuit)

In [435]:
# simulator = cirq.Simulator()
# circuit = cirq.Circuit()    
# circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3))
# circuit.append(cirq.measure(*qubits, key='x'))
# results = simulator.run(circuit, repetitions=100)
# print(results.histogram(key='x'))

In [436]:
# def energy_func(length, h, jr, jc):
#     def energy(measurements):
#         # Reshape measurement into array that matches grid shape.
#         meas_list_of_lists = [measurements[i * length:(i + 1) * length]
#                               for i in range(length)]
#         # Convert true/false to +1/-1.
#         pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

#         tot_energy = np.sum(pm_meas * h)
#         for i, jr_row in enumerate(jr):
#             for j, jr_ij in enumerate(jr_row):
#                 tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
#         for i, jc_row in enumerate(jc):
#             for j, jc_ij in enumerate(jc_row):
#                 tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
#         return tot_energy
#     return energy
# print(results.histogram(key='x', fold_func=energy_func(3, h, jr, jc)))

In [437]:
# def obj_func(result):
#     energy_hist = result.histogram(key='x', fold_func=energy_func(3, h, jr, jc))
#     return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions
# print('Value of the objective function {}'.format(obj_func(results)))

In [438]:
# circuit = cirq.Circuit()
# alpha = sympy.Symbol('alpha')
# beta = sympy.Symbol('beta')
# gamma = sympy.Symbol('gamma')
# circuit.append(one_step(h, jr, jc, alpha, beta, gamma))
# circuit.append(cirq.measure(*qubits, key='x'))
# print(circuit)

In [439]:
# resolver = cirq.ParamResolver({'alpha': 0.1, 'beta': 0.3, 'gamma': 0.7})
# resolved_circuit = cirq.resolve_parameters(circuit, resolver)

# sweep = (cirq.Linspace(key='alpha', start=0.1, stop=0.9, length=5)
#          * cirq.Linspace(key='beta', start=0.1, stop=0.9, length=5)
#          * cirq.Linspace(key='gamma', start=0.1, stop=0.9, length=5))
# results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
# for result in results:
#     print(result.params.param_dict, obj_func(result))

In [440]:
# sweep_size = 10
# sweep = (cirq.Linspace(key='alpha', start=0.0, stop=1.0, length=10)
#          * cirq.Linspace(key='beta', start=0.0, stop=1.0, length=10)
#          * cirq.Linspace(key='gamma', start=0.0, stop=1.0, length=10))
# results = simulator.run_sweep(circuit, params=sweep, repetitions=100)

# min = None
# min_params = None
# for result in results:
#     value = obj_func(result)
#     if min is None or value < min:
#         min = value
#         min_params = result.params
# print('Minimum objective value is {}.'.format(min))